# Using a Møller-Plesset-informed initial point

## Introduction

In this tutorial we are going to describe how to use a Møller-Plesset-informed initial point for the Variational Quantum Eigensolver (VQE) with the ground state calculation interface of Qiskit Nature. The goal is to demonstrate that using such an initial point enables us to compute the ground state of an electronic molecular Hamiltonian using fewer evaluations than the default Hartree-Fock initial point. It will also serve to illustrate some changes to how initial points are 

Please see the tutorial on Ground State Solvers to learn the fundamentals of computing the molecular ground state with Qiskit Nature. To know more about the preparation of the Hamiltonian, check out the Electronic structure and Vibrational structure tutorials. 

## The molecule

As usual, the first step is to define the molecular system. In the following, we obtain the electronic part of a hydrogen molecule.

In [1]:
from qiskit_nature.settings import settings
settings.dict_aux_operators = True

from qiskit_nature.second_q.drivers import UnitsType, Molecule
from qiskit_nature.second_q.drivers import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.second_q.problems import ElectronicStructureProblem

molecule = Molecule(
    geometry=[["Li", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 1.596]]], charge=0, multiplicity=1
)

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

# freeze_core = FreezeCoreTransformer()

problem = ElectronicStructureProblem(driver)#, transformers=[freeze_core])

## The ansatz

In [2]:
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.mappers import QubitConverter, JordanWignerMapper, ParityMapper


# generate the second-quantized operators
main_op, _ = problem.second_q_ops()

grouped_property = problem.grouped_property_transformed

particle_number = grouped_property.get_property(ParticleNumber)
num_particles = (particle_number.num_alpha, particle_number.num_beta)
num_spin_orbitals = particle_number.num_spin_orbitals

# setup the qubit converter
qubit_converter = QubitConverter(JordanWignerMapper())

# map to qubit operators
qubit_op = qubit_converter.convert(main_op, num_particles=num_particles)

# setup the ansatz for VQE
ansatz = UCCSD(
    qubit_converter=qubit_converter,
    num_particles=num_particles,
    num_spin_orbitals=num_spin_orbitals,
)


## The initial point

In [3]:
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint

hf_initial_point = HFInitialPoint()
hf_initial_point.grouped_property = grouped_property
hf_initial_point.ansatz = ansatz
hf_initial_point_np = hf_initial_point.to_numpy_array()

In [4]:
from qiskit_nature.second_q.algorithms.initial_points import MP2InitialPoint

mp2_initial_point = MP2InitialPoint()
mp2_initial_point.grouped_property = grouped_property
mp2_initial_point.ansatz = ansatz
mp2_initial_point_np = mp2_initial_point.to_numpy_array()

## The Solver

Then we need to define a solver. The solver is the algorithm through which the ground state is computed.

To define the VQE solver one needs two essential elements:

1. A variational form: here we use the Unitary Coupled Cluster (UCC) ansatz (see for instance [Physical Review A 98.2 (2018): 022322]). Since it is a chemistry standard, a factory is already available allowing a fast initialization of a VQE with UCC. The default is to use all single and double excitations. However, the excitation type (S, D, SD) as well as other parameters can be selected.
2. An initial state: the initial state of the qubits. In the factory used above, the qubits are initialized in the Hartree-Fock (see the electronic structure tutorial) initial state (the qubits corresponding to occupied MOs are $|1\rangle$ and those corresponding to virtual MOs are $|0\rangle$.
3. The backend: this is the quantum machine on which the right part of the figure above will be performed. Here we ask for the perfect quantum emulator (```aer_simulator_statevector```). 

One could also use any available ansatz / initial state or even define one's own. For instance,



We will be using the Variational Quantum Eigensolver (VQE) algorithm. The VQE algorithms works by exchanging information between a classical and a quantum computer as depicted in the following figure.

<img src="aux_files/vqe.png" width="600">

Let's initialize a VQE solver.

In [5]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import L_BFGS_B
from qiskit.algorithms import VQE

optimizer = L_BFGS_B()

quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))
hf_vqe = VQE(
    ansatz, optimizer=optimizer, quantum_instance=quantum_instance, initial_point=hf_initial_point_np
)

Let's initialize a VQE solver using an `MP2InitialPoint`.

In [6]:
mp2_vqe = VQE(
    ansatz, optimizer=optimizer, quantum_instance=quantum_instance, initial_point=mp2_initial_point_np
)

## The calculation and results

We are now ready to run the calculation.

In [7]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

hf_gse = GroundStateEigensolver(qubit_converter, hf_vqe)
hf_result = hf_gse.solve(problem)
print(hf_result)

/Users/declanmillar/Projects/qiskit/qiskit-nature/.venv/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/declanmillar/Projects/qiskit/qiskit-nature/.venv/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): 0.0
  - computed part:      0.0
~ Nuclear repulsion energy (Hartree): 0.994694005489
> Total ground state energy (Hartree): 0.994694005489
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 0.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  3.01600289]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  3.01600289]  Total: 3.01600289
                 (debye): [0.0  0.0  7.66591398]  Total: 7.66591398
 


In [8]:
print(hf_vqe._ret)

{   'aux_operator_eigenvalues': {   'AngularMomentum': (0.0, 0.0),
                                    'DipoleMomentX': (0.0, 0.0),
                                    'DipoleMomentY': (0.0, 0.0),
                                    'DipoleMomentZ': (-0.0, 0.0),
                                    'Magnetization': (0.0, 0.0),
                                    'ParticleNumber': (0.0, 0.0)},
    'cost_function_evals': 93,
    'eigenstate': array([1.+1.59872116e-14j, 0.+0.00000000e+00j, 0.+0.00000000e+00j, ...,
       0.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j]),
    'eigenvalue': (-1.318e-15+0j),
    'optimal_parameters': {   ParameterVectorElement(t[4]): 0.0,
                              ParameterVectorElement(t[5]): 0.0,
                              ParameterVectorElement(t[3]): 0.0,
                              ParameterVectorElement(t[2]): 0.0,
                              ParameterVectorElement(t[1]): 0.0,
                              ParameterVectorElement(t[

In [9]:
mp2_gse = GroundStateEigensolver(qubit_converter, mp2_vqe)
mp2_result = mp2_gse.solve(problem)
print(mp2_result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): 0.0
  - computed part:      0.0
~ Nuclear repulsion energy (Hartree): 0.994694005489
> Total ground state energy (Hartree): 0.994694005489
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 0.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  3.01600289]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  3.01600289]  Total: 3.01600289
                 (debye): [0.0  0.0  7.66591398]  Total: 7.66591398
 


In [10]:
print(mp2_vqe._ret)

{   'aux_operator_eigenvalues': {   'AngularMomentum': (0.0, 0.0),
                                    'DipoleMomentX': (0.0, 0.0),
                                    'DipoleMomentY': (0.0, 0.0),
                                    'DipoleMomentZ': (0.0, 0.0),
                                    'Magnetization': (0.0, 0.0),
                                    'ParticleNumber': (0.0, 0.0)},
    'cost_function_evals': 93,
    'eigenstate': array([ 1.00000000e+00+3.70443190e-14j, -9.83761056e-16-3.11319895e-15j,
       -6.91821990e-16-3.12546530e-15j, ...,
        1.79352226e-33+7.64353447e-33j, -1.34815096e-32+3.56297040e-33j,
       -7.12383666e-33+3.09525295e-33j]),
    'eigenvalue': (4.108e-15+0j),
    'optimal_parameters': {   ParameterVectorElement(t[4]): 0.0,
                              ParameterVectorElement(t[5]): 0.0,
                              ParameterVectorElement(t[3]): 0.0,
                              ParameterVectorElement(t[2]): 0.0,
                              